<a href="https://colab.research.google.com/github/kekesampaio/AluraIA/blob/main/Projeto_K_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%pip -q install google-genai

In [13]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [14]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [15]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################


# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

NameError: name 'resposta' is not defined

In [16]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no [Instagram](https://www.instagram.com/aluraonline/), [LinkedIn](https://www.linkedin.com/school/aluraonline/), [Twitter](https://twitter.com/AluraOnline) e outras plataformas, pois eles costumam anunciar os eventos por lá.
*   **Verificar a página de eventos da Alura:** Consulte a página de eventos da Alura no site oficial deles. Lá, você pode encontrar informações sobre os próximos eventos e imersões.

Espero que isso ajude você a encontrar a data da próxima Imersão IA com Google Gemini!

In [17]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

AttributeError: 'NoneType' object has no attribute 'web_search_queries'

In [18]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [19]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [20]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [21]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [22]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca informações no Google",
        tools=[google_search],
        instruction="""
        Você é um assitente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search)
        para recuperar a ultimas noticias de lançamento muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamento relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias, é possível que ele não seja tão relevante assim
        e pode ser substituido por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [23]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""Você é um planejador de conteudo, especialista em redes sociais e criação de sites. Com base na lista de lançamentos mais recentes e relevantes no buscador vco~e deve:
        usar a ferramenta de busca do google (google_search) para criar um plano sobre quais são os pontos mais relevantes que poderiamos abordar em um post ou na apresentação de um site sobre cada um deles.
        Você também pode usar o (google search) para encontrar mais informações sobre os temas a aprofundar.
        Ao final, você ira escolher o tema mais relevante entre eles com base na suas pesquisas e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem abordados no post que será escrito ou no site que será desenvolvindo posteriormente.
        """,
        description="Agente que planeja posts e sites",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [24]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais e criação de sites.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado ou desenvolver um site.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagrame criador de site"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [27]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram e sites de marketing.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

if not topico:
  print("Você esqueceu de digitar o topico")
else:
  print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- Resultado do agente 1 (Buscador)---\n")
  display(to_markdown(lancamentos_buscados))
  print("-----------------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- Resultado do agente 2 (Planejador)---\n")
  display(to_markdown(plano_de_post))
  print("-----------------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- Resultado do agente 3 (Redator)---\n")
  display(to_markdown(rascunho_de_post))
  print("-----------------------------------------------------------")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n--- Resultado do agente 4 (Revisor)---\n")
  display(to_markdown(post_final))
  print("-----------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
